In [2]:
#required library
from music21 import *
import music21
import os
import glob
import re
import numpy as np
import json

In [3]:
#key transpose profile
MtoC={
    'Gb':-6,
    'F':-5,
    'E':-4,
    'D#':-3,
    'Eb':-3,
    'D':-2,
    'C#':-1,
    'Db':-1,
    'C':0,    
    'B':1,
    'A#':2,
    'Bb':2,
    'A':3,
    'G#':4,
    'Ab':4,
    'G':5,
    'F#':6
}
mtoA={
    'D#':6,
    'G#':5,
    'Ab':5,
    'C#':4,
    'F#':3,
    'B':2,
    'E':1,
    'A':0,
    'D':-1,
    'G':-2,
    'C':-3,
    'F':-4,
    'Bb':-5,
    'Eb':-6
}
threshold=9

In [4]:
#read file
c = converter.parse('../musicxml(ok)/Étude_in_Gb_Major_Op25.mxl',makeNotation=False)

In [5]:
#remove key signature
for e in c.recurse().getElementsByClass(key.KeySignature):
    while e.sharps!=0:
        e.transpose(-1,inPlace=True)

In [ ]:
#remove key signature
for e in c.recurse().getElementsByClass(stream):
    print(e)

In [ ]:
def cal_offset(e):
    if e is None:
        return 0
    return e.offset+cal_offset(e.activeSite)

In [10]:
#extract notes
all_notes=[]
for el in c.recurse().notes:
    if el.lyric is not None:
        el.lyric=el.lyric.replace('♭','b')
    
    #print(el)
    #print([el.getOffsetBySite(c),el.lyric, el, el.activeSite,el.offset, el.activeSite.offset])
    all_notes.append([el.lyric, el, cal_offset(el)])
    
    #DEBUG
    #print([el.lyric, el, cal_offset(el)])

In [11]:
#sort by first occurence
b = sorted(all_notes, key=lambda x: x[0] if x[0] is not None else "ZZZ")
b=sorted(all_notes,key=lambda x: x[-1])


#DEBUG
for e in b:
    print(e)

['GbM(I)', <music21.note.Note B->, 0.0]
[None, <music21.note.Note B->, 0.0]
[None, <music21.note.Note G->, 0.0]
[None, <music21.chord.Chord G-5 B-5>, 0.25]
[None, <music21.chord.Chord C-5 C-6>, 0.5]
[None, <music21.chord.Chord D-4 G-4>, 0.5]
[None, <music21.chord.Chord C5 C6>, 0.75]
['V', <music21.note.Note D->, 1.0]
[None, <music21.note.Note D->, 1.0]
[None, <music21.note.Note F>, 1.0]
[None, <music21.chord.Chord A-5 D-6>, 1.25]
[None, <music21.chord.Chord E-5 E-6>, 1.5]
[None, <music21.chord.Chord A-3 D-4 A-4>, 1.5]
[None, <music21.chord.Chord D-5 D-6>, 1.75]
['VI', <music21.note.Note G->, 2.0]
[None, <music21.note.Note G->, 2.0]
[None, <music21.note.Note E->, 2.0]
[None, <music21.chord.Chord E-5 G-5>, 2.25]
[None, <music21.chord.Chord A-4 A-5>, 2.5]
[None, <music21.chord.Chord G-3 B-3 E-4>, 2.5]
[None, <music21.chord.Chord A4 A5>, 2.75]
['III', <music21.note.Note B->, 3.0]
[None, <music21.note.Note B->, 3.0]
[None, <music21.note.Note D->, 3.0]
[None, <music21.chord.Chord F5 B-5>, 3.

In [12]:
k = None
previous_change_offset = 0
previous_change_index = 0
for idx, e in enumerate(reversed(b)):
    if e[0] is None or "(" not in e[0]:
        # label without key
        e.append(0)
    else:
        # label with key
        if k is None:
            # initial condition
            k = e[0].split("(")[0]
            e.append(b[-1][2] - e[2])
            previous_change_offset = e[2]
            previous_change_index = idx
        else:
            # changed key
            if k != e[0].split("(")[0]:
                e.append(previous_change_offset - e[2])
                previous_change_offset = e[2]
                previous_change_index = idx
                k = e[0].split("(")[0]
            # no key change
            else:
                # trace back
                e.append(
                    previous_change_offset
                    - e[2]
                    + b[-previous_change_index - 1][-1]
                )
                b[-previous_change_index - 1][-1] = 0
                previous_change_offset = e[2]
                previous_change_index = idx

In [13]:
current_key = None
Major = None
for e in b:
    if current_key is None:
        # fail-safe (idk wt is this for, seems just some empty meaningless thing ....)
        if "NoChord" in e[1].classes:
            continue

        if e[1].lyric is None:
            # infer key
            for i in b:
                if i[0] is not None:
                    assert "(" in i[0]
                    e[1].addLyric('infer '+i[0].split('(')[0])
                    current_key = i[0]
                    break
            else:
                assert 1 == 0  # no key marked???
        else:
            current_key = e[1].lyric

        print("start")
        e[1].addLyric('start')#every score should veriyf its start***

        # identify major or minor
        if "M" in current_key:
            Major = True
        else:
            Major = False

        # identify key name
        if Major:
            e[1].addLyric('trans->CM')
            current_key = current_key.split("M")[0]
        else:
            e[1].addLyric('trans->Am')
            current_key = current_key.split("m")[0]

    # check if key change
    elif (
        e[1].lyric is not None
        and ("m" in e[1].lyric or "M" in e[1].lyric)
        and "(" in e[1].lyric
    ):  # '(' --> is just to make sure it is key change

        # DEBUG
        # print(current_key,Major)
        # print(e[1].lyric)
        # print('----------------------')

        # check if key is different from previous key
        if (
            Major and "M" in e[1].lyric and current_key == e[1].lyric.split("M")[0]
        ) or (
            not Major
            and "m" in e[1].lyric
            and current_key == e[1].lyric.split("m")[0]
        ):

             e[1].addLyric("same key as previous change")
        else:
            # update key
            if e[-1] < threshold:
                pass
                e[1].addLyric("2nd dominant")
            else:
                print("change key")
                current_key = e[1].lyric

                # identify major or minor
                if "M" in current_key:
                    Major = True
                else:
                    Major = False

                # identify key name
                if Major:
                    current_key = current_key.split("M")[0]
                    e[1].addLyric("trans->CM")
                else:
                    current_key = current_key.split("m")[0]
                    e[1].addLyric("trans->Am")

    if Major:
        e[1].transpose(MtoC[current_key], inPlace=True)
    else:
        e[1].transpose(mtoA[current_key], inPlace=True)

start
change key
change key


In [14]:
post=c.flattenParts().flat
for note in post.notes:
    print(note,note.offset,note.lyric)

<music21.note.Note E> 0.0 GbM(I)
start
trans->CM
<music21.note.Note E> 0.0 None
<music21.note.Note C> 0.0 None
<music21.chord.Chord C5 E5> 0.25 None
<music21.chord.Chord F4 F5> 0.5 None
<music21.chord.Chord G3 C4> 0.5 None
<music21.chord.Chord F#4 F#5> 0.75 None
<music21.note.Note G> 1.0 V
<music21.note.Note G> 1.0 None
<music21.note.Note B> 1.0 None
<music21.chord.Chord D5 G5> 1.25 None
<music21.chord.Chord A4 A5> 1.5 None
<music21.chord.Chord D3 G3 D4> 1.5 None
<music21.chord.Chord G4 G5> 1.75 None
<music21.note.Note C> 2.0 VI
<music21.note.Note C> 2.0 None
<music21.note.Note A> 2.0 None
<music21.chord.Chord A4 C5> 2.25 None
<music21.chord.Chord D4 D5> 2.5 None
<music21.chord.Chord C3 E3 A3> 2.5 None
<music21.chord.Chord E-4 E-5> 2.75 None
<music21.note.Note E> 3.0 III
<music21.note.Note E> 3.0 None
<music21.note.Note G> 3.0 None
<music21.chord.Chord B4 E5> 3.25 None
<music21.chord.Chord F4 F5> 3.5 None
<music21.chord.Chord B2 E3 B3> 3.5 None
<music21.chord.Chord E4 E5> 3.75 None
<mu

In [17]:
#save

# SX = musicxml.m21ToXml.ScoreExporter(c)
# mxScore = SX.parse()
# SX.dump(mxScore)
GEX = musicxml.m21ToXml.GeneralObjectExporter(c)
out = GEX.parse()
with open("test.musicxml", "wb") as f:
        f.write(out)

MusicXMLExportException: In part (Piano), measure (1): Attempted to create a second <staff> tag

In [33]:
piece = "test.musicxml"
c = converter.parse(piece)
partStream = c.parts.stream()
post = c.flattenParts().flat
all_notes = []
for note in post.notes:
    print(note,note.offset,note.lyric)

<music21.note.Note C> -1.5 None
<music21.chord.Chord G3 C4> -1.0 None
<music21.note.Note B> -0.5 None
<music21.note.Note E> 0.0 GbM(I)
start
trans->CM
<music21.note.Note E> 0.0 None
<music21.chord.Chord D3 G3 D4> 0.0 None
<music21.chord.Chord C5 E5> 0.25 None
<music21.chord.Chord F4 F5> 0.5 None
<music21.note.Note A> 0.5 None
<music21.chord.Chord F#4 F#5> 0.75 None
<music21.note.Note G> 1.0 V
<music21.note.Note G> 1.0 None
<music21.chord.Chord C3 E3 A3> 1.0 None
<music21.chord.Chord D5 G5> 1.25 None
<music21.chord.Chord A4 A5> 1.5 None
<music21.note.Note G> 1.5 None
<music21.chord.Chord G4 G5> 1.75 None
<music21.note.Note C> 2.0 VI
<music21.note.Note C> 2.0 None
<music21.chord.Chord B2 E3 B3> 2.0 None
<music21.chord.Chord A4 C5> 2.25 None
<music21.chord.Chord D4 D5> 2.5 None
<music21.note.Note F> 2.5 None
<music21.chord.Chord E-4 E-5> 2.75 None
<music21.note.Note E> 3.0 III
<music21.note.Note E> 3.0 None
<music21.chord.Chord C3 F3> 3.0 None
<music21.chord.Chord B4 E5> 3.25 None
<music2